In [150]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from gurobipy import *


In [151]:
distances = pd.read_excel('data/distances.xlsx', index_col=0)
distances = distances.fillna(0)
distances = distances + distances.T
distances -= np.eye(distances.shape[0]) * 3


events18 = pd.read_csv('data/events18.csv',sep='\t',index_col=0)
events18["Start"] = pd.to_datetime(events18["Start"])
events18["End"] = pd.to_datetime(events18["End"])

events19 = pd.read_csv('data/events19.csv',sep='\t',index_col=0)
events19["Start"] = pd.to_datetime(events19["Start"])
events19["End"] = pd.to_datetime(events19["End"])

events20 = pd.read_csv('data/events20.csv',sep='\t',index_col=0)
events20["Start"] = pd.to_datetime(events20["Start"])
events20["End"] = pd.to_datetime(events20["End"])

events21 = pd.read_csv('data/events21.csv',sep='\t',index_col=0)
events21["Start"] = pd.to_datetime(events21["Start"])
events21["End"] = pd.to_datetime(events21["End"])

sallesDeCours = pd.read_csv('data/SallesDeCours.csv',sep=';')

students18 = json.load(open('data/students18.json'))
students19 = json.load(open('data/students19.json'))
students20 = json.load(open('data/students20.json'))
students21 = json.load(open('data/students21.json'))

In [152]:
# Function to check if two intervals intersect
def time_intersect(int1,int2):
    if int1[0] < int2[0] < int1[1] or int1[0] < int2[1] < int1[1]: # event 2 starts or ends during event 1
        return 1
    elif int2[0] < int1[0] < int2[1] or int2[0] < int1[1] < int2[1]: # event 1 starts or ends during event 2
        return 1
    else:
        return 0

# function to create a dictionnary with students as keys and list of events as values
def get_courses(students, events):
    courses = {}
    for event in students:
        for student in students[event]:
            if student not in courses:
                courses[student] = [event]
            else:
                courses[student].append(event)

    for student in courses:
        courses[student] = sorted(courses[student], key=lambda x: events.loc[x]["Start"])
        
    return courses

# function to create a pandas dataframe with salles names as index and columns and the distance between them as values
def get_full_distances(distances, salles):
    # create dataframe with all distances between all rooms 
    full_distances = np.zeros((len(salles), len(salles)))
    for i in range(len(salles)):
        for j in range(len(salles)):
            if salles["Building"][i] == salles["Building"][j]:
                full_distances[i,j] = 0
            else:
                full_distances[i,j] = distances.loc[salles["Name"][i][0], salles["Name"][j][0]]
    full_distances = pd.DataFrame(full_distances, index=salles["Name"], columns=salles["Name"])

    return full_distances

# function to compute the number of students moving from one event to another 
def get_students_flow(courses, events):
    flows = np.zeros((len(events), len(events)))
    flows = pd.DataFrame(flows, index=events.index, columns=events.index)
    for course in courses:
        for i in range(len(courses[course])-1):
            flows.loc[courses[course][i], courses[course][i+1]] += 1
    return flows
    


In [153]:
courses18 = get_courses(students18, events18)
courses19 = get_courses(students19, events19)
courses20 = get_courses(students20, events20)
courses21 = get_courses(students21, events21)

In [154]:
full_distances = get_full_distances(distances, sallesDeCours)

In [155]:
flows18 = get_students_flow(courses18, events18)
flows19 = get_students_flow(courses19, events19)
flows20 = get_students_flow(courses20, events20)
flows21 = get_students_flow(courses21, events21)

In [156]:
m = Model("room_allocation")

Set parameter Username


GurobiError: Version number is 10.0, license is for version 9.0